# Trajectories (Coppelia Sim)

## 3.

Loading remote API

In [7]:
# Import the python library that connects to CoppeliaSim, the file is sim.py.
try:
    import sim
except:
    print('--------------------------------------------------------------')
    print('"sim.py" could not be imported. This means very probably that')
    print('either "sim.py" or the remoteApi library could not be found.')
    print('Make sure both are in the same folder as this file,')
    print('or appropriately adjust the file "sim.py"')
    print('--------------------------------------------------------------')
    print('')

import time

%pylab inline

Populating the interactive namespace from numpy and matplotlib


Checking if python connects to Coppelia

In [8]:
print('Program started')
sim.simxFinish(-1)  # just in case, close all opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected to remote API server')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)

    # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive. You can guarantee this with (for example):
    sim.simxGetPingTime(clientID)

    # Now close the connection to CoppeliaSim:
    sim.simxFinish(clientID)
else:
    print('Failed connecting to remote API server')
print('Program ended')

Program started
Connected to remote API server
Program ended


## Robot

In [9]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(clientID, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)

### Controlling the Omnirobot

In [18]:
# Joint name in coppelia
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
r = robot('Omnirob', motor_names)  # Create an instance of our robot

# Initial locations -> Going to SPHERE1
print('Moving to Sphere1')
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

desired_position = r.get_object_position('Sphere1')
print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

relative_position = r.get_object_relative_position('Sphere1')
print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))
ii
# Time duration
tf = 15
# Time interval
time_steps = linspace(0, tf, 500)

a1 = (desired_position - robot_position) / tf
a0 = robot_position

for t in time_steps:
    point_traj = a1 * t + a0
    vel_traj = a1
    print(point_traj)
    
    
    # Sensing
    robot_position = r.get_position()
    
    # Trajectory tracker
    u = 10 * (point_traj - robot_position) + vel_traj
    
    vx, vy, vz = u
    
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    
    time.sleep(tf/200)
    
# SPHERE0
print('Moving to Sphere0')
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

desired_position = r.get_object_position('Sphere0')
print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

relative_position = r.get_object_relative_position('Sphere0')
print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

# Time duration
tf = 15
# Time interval
time_steps = linspace(0, tf, 500)

a1 = (desired_position - robot_position) / tf
a0 = robot_position

for t in time_steps:
    point_traj = a1 * t + a0
    vel_traj = a1
    print(point_traj)
    
    # Sensing
    robot_position = r.get_position()
    
    # Trajectory tracker
    u = 10 * (point_traj - robot_position) + vel_traj
    
    vx, vy, vz = u
    
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    
    time.sleep(tf/200)

# SPHERE
print('Moving to Sphere')
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

desired_position = r.get_object_position('Sphere')
print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

relative_position = r.get_object_relative_position('Sphere')
print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

# Time duration
tf = 15
# Time interval
time_steps = linspace(0, tf, 500)

a1 = (desired_position - robot_position) / tf
a0 = robot_position

for t in time_steps:
    point_traj = a1 * t + a0
    vel_traj = a1
    print(point_traj)
    
    # Sensing
    robot_position = r.get_position()
    
    # Trajectory tracker
    u = 10 * (point_traj - robot_position) + vel_traj
    
    vx, vy, vz = u
    
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    
    time.sleep(tf/200)

# SPHERE2
print('Moving to Sphere2')
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

desired_position = r.get_object_position('Sphere2')
print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

relative_position = r.get_object_relative_position('Sphere2')
print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

# Time duration
tf = 15
# Time interval
time_steps = linspace(0, tf, 500)

a1 = (desired_position - robot_position) / tf
a0 = robot_position

for t in time_steps:
    point_traj = a1 * t + a0
    vel_traj = a1
    print(point_traj)
    
    # Sensing
    robot_position = r.get_position()
    
    # Trajectory tracker
    u = 10 * (point_traj - robot_position) + vel_traj
    
    vx, vy, vz = u
    
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    
    time.sleep(tf/200)

# SPHERE3
print('Moving to Sphere3')
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

desired_position = r.get_object_position('Sphere3')
print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

relative_position = r.get_object_relative_position('Sphere3')
print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

# Time duration
tf = 15
# Time interval
time_steps = linspace(0, tf, 500)

a1 = (desired_position - robot_position) / tf
a0 = robot_position

for t in time_steps:
    point_traj = a1 * t + a0
    vel_traj = a1
    print(point_traj)
    
    # Sensing
    robot_position = r.get_position()
    
    # Trajectory tracker
    u = 10 * (point_traj - robot_position) + vel_traj
    
    vx, vy, vz = u
    
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    
    time.sleep(tf/200)

# SPHERE0
print('Moving to Sphere4')
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

desired_position = r.get_object_position('Sphere4')
print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

relative_position = r.get_object_relative_position('Sphere4')
print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

# Time duration
tf = 15
# Time interval
time_steps = linspace(0, tf, 500)

a1 = (desired_position - robot_position) / tf
a0 = robot_position

for t in time_steps:
    point_traj = a1 * t + a0
    vel_traj = a1
    print(point_traj)
    
    # Sensing
    robot_position = r.get_position()
    
    # Trajectory tracker
    u = 10 * (point_traj - robot_position) + vel_traj
    
    vx, vy, vz = u
    
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    
    time.sleep(tf/200)

r.close_connection() 

Robot connected
Moving to Sphere1
Robot position: (7.62, 8.55) 
Desired position: (0.40, 8.05) 
Relative position: (-7.22, -0.50) 
[7.62499475 8.54970646 0.35449859]
[7.61051581 8.54870504 0.35388838]
[7.59603687 8.54770362 0.35327816]
[7.58155792 8.54670221 0.35266794]
[7.56707898 8.54570079 0.35205772]
[7.55260003 8.54469937 0.35144751]
[7.53812109 8.54369796 0.35083729]
[7.52364214 8.54269654 0.35022707]
[7.5091632  8.54169512 0.34961685]
[7.49468426 8.5406937  0.34900664]
[7.48020531 8.53969229 0.34839642]
[7.46572637 8.53869087 0.3477862 ]
[7.45124742 8.53768945 0.34717598]
[7.43676848 8.53668803 0.34656577]
[7.42228953 8.53568662 0.34595555]
[7.40781059 8.5346852  0.34534533]
[7.39333165 8.53368378 0.34473511]
[7.3788527  8.53268237 0.3441249 ]
[7.36437376 8.53168095 0.34351468]
[7.34989481 8.53067953 0.34290446]
[7.33541587 8.52967811 0.34229424]
[7.32093692 8.5286767  0.34168402]
[7.30645798 8.52767528 0.34107381]
[7.29197903 8.52667386 0.34046359]
[7.27750009 8.52567244 0.3398

[4.26587967 8.31737765 0.21292811]
[4.25140072 8.31637623 0.21231789]
[4.23692178 8.31537482 0.21170767]
[4.22244283 8.3143734  0.21109745]
[4.20796389 8.31337198 0.21048724]
[4.19348494 8.31237056 0.20987702]
[4.179006   8.31136915 0.2092668 ]
[4.16452706 8.31036773 0.20865658]
[4.15004811 8.30936631 0.20804637]
[4.13556917 8.30836489 0.20743615]
[4.12109022 8.30736348 0.20682593]
[4.10661128 8.30636206 0.20621571]
[4.09213233 8.30536064 0.20560549]
[4.07765339 8.30435923 0.20499528]
[4.06317445 8.30335781 0.20438506]
[4.0486955  8.30235639 0.20377484]
[4.03421656 8.30135497 0.20316462]
[4.01973761 8.30035356 0.20255441]
[4.00525867 8.29935214 0.20194419]
[3.99077972 8.29835072 0.20133397]
[3.97630078 8.2973493  0.20072375]
[3.96182184 8.29634789 0.20011354]
[3.94734289 8.29534647 0.19950332]
[3.93286395 8.29434505 0.1988931 ]
[3.918385   8.29334364 0.19828288]
[3.90390606 8.29234222 0.19767267]
[3.88942711 8.2913408  0.19706245]
[3.87494817 8.29033938 0.19645223]
[3.86046922 8.289337

[0.86332774 8.08204459 0.06952696]
[0.8488488  8.08104317 0.06891675]
[0.83436986 8.08004176 0.06830653]
[0.81989091 8.07904034 0.06769631]
[0.80541197 8.07803892 0.06708609]
[0.79093302 8.0770375  0.06647588]
[0.77645408 8.07603609 0.06586566]
[0.76197513 8.07503467 0.06525544]
[0.74749619 8.07403325 0.06464522]
[0.73301725 8.07303183 0.06403501]
[0.7185383  8.07203042 0.06342479]
[0.70405936 8.071029   0.06281457]
[0.68958041 8.07002758 0.06220435]
[0.67510147 8.06902617 0.06159414]
[0.66062252 8.06802475 0.06098392]
[0.64614358 8.06702333 0.0603737 ]
[0.63166464 8.06602191 0.05976348]
[0.61718569 8.0650205  0.05915327]
[0.60270675 8.06401908 0.05854305]
[0.5882278  8.06301766 0.05793283]
[0.57374886 8.06201624 0.05732261]
[0.55926991 8.06101483 0.05671239]
[0.54479097 8.06001341 0.05610218]
[0.53031202 8.05901199 0.05549196]
[0.51583308 8.05801058 0.05488174]
[0.50135414 8.05700916 0.05427152]
[0.48687519 8.05600774 0.05366131]
[0.47239625 8.05500632 0.05305109]
[0.4579173  8.054004

[-1.0079538   6.27217914  0.2391693 ]
[-1.01623459  6.26291404  0.23855907]
[-1.02451538  6.25364894  0.23794885]
[-1.03279617  6.24438384  0.23733863]
[-1.04107695  6.23511874  0.2367284 ]
[-1.04935774  6.22585364  0.23611818]
[-1.05763853  6.21658854  0.23550796]
[-1.06591932  6.20732344  0.23489773]
[-1.07420011  6.19805834  0.23428751]
[-1.0824809   6.18879324  0.23367729]
[-1.09076169  6.17952814  0.23306706]
[-1.09904248  6.17026304  0.23245684]
[-1.10732327  6.16099794  0.23184661]
[-1.11560406  6.15173284  0.23123639]
[-1.12388484  6.14246774  0.23062617]
[-1.13216563  6.13320264  0.23001594]
[-1.14044642  6.12393754  0.22940572]
[-1.14872721  6.11467244  0.2287955 ]
[-1.157008    6.10540734  0.22818527]
[-1.16528879  6.09614224  0.22757505]
[-1.17356958  6.08687714  0.22696483]
[-1.18185037  6.07761204  0.2263546 ]
[-1.19013116  6.06834694  0.22574438]
[-1.19841194  6.05908184  0.22513416]
[-1.20669273  6.04981674  0.22452393]
[-1.21497352  6.04055164  0.22391371]
[-1.22325431

[-2.804885    4.26165247  0.10675079]
[-2.81316579  4.25238737  0.10614057]
[-2.82144658  4.24312227  0.10553034]
[-2.82972737  4.23385717  0.10492012]
[-2.83800816  4.22459207  0.1043099 ]
[-2.84628895  4.21532697  0.10369967]
[-2.85456974  4.20606187  0.10308945]
[-2.86285053  4.19679677  0.10247922]
[-2.87113132  4.18753167  0.101869  ]
[-2.87941211  4.17826657  0.10125878]
[-2.88769289  4.16900147  0.10064855]
[-2.89597368  4.15973637  0.10003833]
[-2.90425447  4.15047127  0.09942811]
[-2.91253526  4.14120617  0.09881788]
[-2.92081605  4.13194107  0.09820766]
[-2.92909684  4.12267598  0.09759744]
[-2.93737763  4.11341088  0.09698721]
[-2.94565842  4.10414578  0.09637699]
[-2.95393921  4.09488068  0.09576677]
[-2.96222     4.08561558  0.09515654]
[-2.97050078  4.07635048  0.09454632]
[-2.97878157  4.06708538  0.0939361 ]
[-2.98706236  4.05782028  0.09332587]
[-2.99534315  4.04855518  0.09271565]
[-3.00362394  4.03929008  0.09210542]
[-3.01190473  4.03002498  0.0914952 ]
[-3.02018552

### Controlling the Quadricopter

In [19]:
# Joint name in coppelia
motor_names = ['Quadricopter_propeller1', 'Quadricopter_propeller2', 'Quadricopter_propeller3', 'Quadricopter_propeller4']
r = robot('Quadricopter_target', motor_names)  # Create an instance of our robot
# Quadricopter_target appears when you start simulation

# Initial locations -> Going to SPHERE6
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

desired_position = r.get_object_position('Sphere6')
print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

relative_position = r.get_object_relative_position('Sphere6')
print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

# Time duration
tf = 15
# Time interval
time_steps = linspace(0, tf, 500)

a1 = (desired_position - robot_position) / tf
a0 = robot_position

for t in time_steps:
    point_traj = a1 * t + a0
    vel_traj = a1
    print(point_traj)
    
    
    # Sensing
    robot_position = r.get_position()
    
    # Trajectory tracker
    u = 10 * (point_traj - robot_position) + vel_traj
    
    vx, vy, vz = u
    
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    
    time.sleep(tf/200)
    
# SPHERE5
print('Moving to Sphere5')
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

desired_position = r.get_object_position('Sphere5')
print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

relative_position = r.get_object_relative_position('Sphere5')
print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

# Time duration
tf = 15
# Time interval
time_steps = linspace(0, tf, 500)

a1 = (desired_position - robot_position) / tf
a0 = robot_position

for t in time_steps:
    point_traj = a1 * t + a0
    vel_traj = a1
    print(point_traj)
    
    # Sensing
    robot_position = r.get_position()
    
    # Trajectory tracker
    u = 10 * (point_traj - robot_position) + vel_traj
    
    vx, vy, vz = u
    
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    
    time.sleep(tf/200)

# SPHERE7
print('Moving to Sphere7')
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

desired_position = r.get_object_position('Sphere7')
print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

relative_position = r.get_object_relative_position('Sphere7')
print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

# Time duration
tf = 15
# Time interval
time_steps = linspace(0, tf, 500)

a1 = (desired_position - robot_position) / tf
a0 = robot_position

for t in time_steps:
    point_traj = a1 * t + a0
    vel_traj = a1
    print(point_traj)
    
    # Sensing
    robot_position = r.get_position()
    
    # Trajectory tracker
    u = 10 * (point_traj - robot_position) + vel_traj
    
    vx, vy, vz = u
    
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    
    time.sleep(tf/200)

# SPHERE10
print('Moving to Sphere10')
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

desired_position = r.get_object_position('Sphere10')
print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

relative_position = r.get_object_relative_position('Sphere10')
print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

# Time duration
tf = 15
# Time interval
time_steps = linspace(0, tf, 500)

a1 = (desired_position - robot_position) / tf
a0 = robot_position

for t in time_steps:
    point_traj = a1 * t + a0
    vel_traj = a1
    print(point_traj)
    
    # Sensing
    robot_position = r.get_position()
    
    # Trajectory tracker
    u = 10 * (point_traj - robot_position) + vel_traj
    
    vx, vy, vz = u
    
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    
    time.sleep(tf/200)

# SPHERE8
print('Moving to Sphere8')
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

desired_position = r.get_object_position('Sphere8')
print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

relative_position = r.get_object_relative_position('Sphere8')
print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

# Time duration
tf = 15
# Time interval
time_steps = linspace(0, tf, 500)

a1 = (desired_position - robot_position) / tf
a0 = robot_position

for t in time_steps:
    point_traj = a1 * t + a0
    vel_traj = a1
    print(point_traj)
    
    # Sensing
    robot_position = r.get_position()
    
    # Trajectory tracker
    u = 10 * (point_traj - robot_position) + vel_traj
    
    vx, vy, vz = u
    
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    
    time.sleep(tf/200)

# SPHERE9
print('Moving to Sphere9')
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

desired_position = r.get_object_position('Sphere9')
print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

relative_position = r.get_object_relative_position('Sphere9')
print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))

# Time duration
tf = 15
# Time interval
time_steps = linspace(0, tf, 500)

a1 = (desired_position - robot_position) / tf
a0 = robot_position

for t in time_steps:
    point_traj = a1 * t + a0
    vel_traj = a1
    print(point_traj)
    
    # Sensing
    robot_position = r.get_position()
    
    # Trajectory tracker
    u = 10 * (point_traj - robot_position) + vel_traj
    
    vx, vy, vz = u
    
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    
    time.sleep(tf/200)

r.close_connection() 

Robot connected
Robot position: (5.87, 9.05) 
Desired position: (0.40, 8.05) 
Relative position: (-5.47, -1.00) 
[5.87499952 9.05000019 0.5       ]
[5.86402758 9.04799618 0.4990982 ]
[5.85305564 9.04599217 0.49819639]
[5.8420837  9.04398816 0.49729459]
[5.83111176 9.04198415 0.49639279]
[5.82013982 9.03998014 0.49549098]
[5.80916788 9.03797613 0.49458918]
[5.79819594 9.03597212 0.49368737]
[5.787224   9.03396811 0.49278557]
[5.77625206 9.0319641  0.49188377]
[5.76528012 9.02996009 0.49098196]
[5.75430818 9.02795608 0.49008016]
[5.74333624 9.02595207 0.48917836]
[5.7323643  9.02394806 0.48827655]
[5.72139236 9.02194405 0.48737475]
[5.71042043 9.01994004 0.48647295]
[5.69944849 9.01793603 0.48557114]
[5.68847655 9.01593202 0.48466934]
[5.67750461 9.01392801 0.48376754]
[5.66653267 9.011924   0.48286573]
[5.65556073 9.00991999 0.48196393]
[5.64458879 9.00791598 0.48106212]
[5.63361685 9.00591197 0.48016032]
[5.62264491 9.00390796 0.47925852]
[5.61167297 9.00190395 0.47835671]
[5.60070103 

[3.32950947 8.58506989 0.29078156]
[3.31853753 8.58306588 0.28987976]
[3.30756559 8.58106187 0.28897796]
[3.29659365 8.57905786 0.28807615]
[3.28562171 8.57705385 0.28717435]
[3.27464977 8.57504984 0.28627255]
[3.26367783 8.57304583 0.28537074]
[3.25270589 8.57104182 0.28446894]
[3.24173395 8.56903781 0.28356713]
[3.23076201 8.5670338  0.28266533]
[3.21979007 8.56502979 0.28176353]
[3.20881813 8.56302578 0.28086172]
[3.19784619 8.56102177 0.27995992]
[3.18687425 8.55901776 0.27905812]
[3.17590231 8.55701375 0.27815631]
[3.16493037 8.55500974 0.27725451]
[3.15395843 8.55300573 0.27635271]
[3.14298649 8.55100172 0.2754509 ]
[3.13201455 8.54899771 0.2745491 ]
[3.12104261 8.5469937  0.27364729]
[3.11007067 8.54498969 0.27274549]
[3.09909873 8.54298568 0.27184369]
[3.0881268  8.54098167 0.27094188]
[3.07715486 8.53897766 0.27004008]
[3.06618292 8.53697365 0.26913828]
[3.05521098 8.53496964 0.26823647]
[3.04423904 8.53296563 0.26733467]
[3.0332671  8.53096162 0.26643287]
[3.02229516 8.528957

KeyboardInterrupt: 